In [27]:
import sys
sys.path.append("/home/wanxinli/EHR-OT/")

from ast import literal_eval
import collections
from mimic_common import *
import os
import pandas as pd
from statistics import mean, stdev

In [28]:
def read_df(group_name, group_1, group_2, suffix = None):
    """ 
    Absoluate bound for filtering out ratios
    """
    target_equity_path = os.path.join(mimic_output_dir, f"exp4_{group_name}_{group_2}2{group_1}_equity.csv")
    if suffix is not None:
        target_equity_path = os.path.join(mimic_output_dir, f"exp4_{group_name}_{group_2}2{group_1}_{suffix}_equity.csv")
    target_equity_df = pd.read_csv(target_equity_path, header=0, index_col=None, converters={'target_codes': literal_eval})

    return target_equity_df

In [45]:
def find_outlier_codes(df):
    """ 
    :param Dataframe df: the read in dataframe

    Find outlier admissions' codes
    """
    diff_percents = list(df['target_diff_percent'])
    thres = mean(diff_percents)+3*stdev(diff_percents)

    outlier_codes = []
    for _, row in df.iterrows():
        if abs(row['target_diff_percent']) > thres:
            outlier_codes.append(row['target_codes'])
    return outlier_codes
    

In [30]:
def determine_code_freq(outlier_codes, occur_percent = 0.2):
    """ 
    Determine code frequencies for codes in outlier_codes
    :param float occur_percent: the percentage out of all outlier admissions to be classified as outlier code
    """
    num_adm = len(outlier_codes)
    outlier_codes = [code for code_vec in outlier_codes for code in code_vec]
    code_freq = collections.Counter(outlier_codes)
    code_freq = dict(sorted(code_freq.items(), key=lambda item: item[1], reverse=True))

    code_freq = {key: value for key, value in code_freq.items() if key != '' and value > occur_percent*num_adm}
    return code_freq


In [31]:
def entire_proc(group_name, group_1, group_2):
    df = read_df(group_name, group_1, group_2)
    outlier_codes = find_outlier_codes(df)
    code_freq = determine_code_freq(outlier_codes)
    for code, freq in code_freq.items():
        chapter_num, chapter_name = decide_ICD_chapter(code)
        title = get_code_title(code)
        print(f"ICD code: {code}, chapter number: {chapter_num}, chapter name: {chapter_name}, code title: {title}")
    return code_freq.keys()


In [39]:
suffix = None
group_name = 'insurance'
groups = ['Self_Pay', 'Private', 'Government', 'Medicare', 'Medicaid']
outlier_codes = []

for i, group_1 in enumerate(groups):
    for j, group_2 in enumerate(groups):
        print(f"group 1 is: {group_1}, group_2 is: {group_2}")
        if i <= j:
            continue
        outlier_codes.extend(entire_proc(group_name, group_1, group_2))

print("========= printing outlier codes =============")
outlier_code_dict = dict(Counter(outlier_codes))
for code, freq in outlier_code_dict.items():
    chapter_num, chapter_name = decide_ICD_chapter(code)
    title = get_code_title(code)
    freq = freq/(len(groups)*(len(groups)-1)/2)
    print(f"ICD code: {code}, chapter number: {chapter_num}, chapter name: {chapter_name}, code title: {title}, frequency: {freq}")

group 1 is: Self_Pay, group_2 is: Self_Pay
group 1 is: Self_Pay, group_2 is: Private
group 1 is: Self_Pay, group_2 is: Government
group 1 is: Self_Pay, group_2 is: Medicare
group 1 is: Self_Pay, group_2 is: Medicaid
group 1 is: Private, group_2 is: Self_Pay
ICD code: 807.09, chapter number: 17, chapter name: Injury and Poisoning, code title: Fx mult ribs NOS-closed
ICD code: 861.21, chapter number: 17, chapter name: Injury and Poisoning, code title: Lung contusion-closed
ICD code: 427.5, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Cardiac arrest
ICD code: E849.5, chapter number: 18, chapter name: Supplementary Classification of External Causes of Injury and Poisoning, code title: Accid on street/highway
ICD code: 864.14, chapter number: 17, chapter name: Injury and Poisoning, code title: Liver lacerat, major-opn
ICD code: 276.2, chapter number: 3, chapter name: Endocrine, Nutritional and Metabolic Diseases, and Immunity Disorders, code title: Acidos

In [46]:
suffix = None
group_name = 'marital_status'
groups = ['SEPARATED', 'DIVORCED', 'MARRIED','WIDOWED', 'SINGLE']
outlier_codes = []

for i, group_1 in enumerate(groups):
    for j, group_2 in enumerate(groups):
        print(f"group 1 is: {group_1}, group_2 is: {group_2}")
        if i <= j:
            continue
        outlier_codes.extend(entire_proc(group_name, group_1, group_2))

print("========= printing outlier codes =============")
outlier_code_dict = dict(Counter(outlier_codes))
for code, freq in outlier_code_dict.items():
    chapter_num, chapter_name = decide_ICD_chapter(code)
    title = get_code_title(code)
    freq = freq/(len(groups)*(len(groups)-1)/2)
    print(f"ICD code: {code}, chapter number: {chapter_num}, chapter name: {chapter_name}, code title: {title}, frequency: {freq}")

group 1 is: SEPARATED, group_2 is: SEPARATED
group 1 is: SEPARATED, group_2 is: DIVORCED
group 1 is: SEPARATED, group_2 is: MARRIED
group 1 is: SEPARATED, group_2 is: WIDOWED
group 1 is: SEPARATED, group_2 is: SINGLE
group 1 is: DIVORCED, group_2 is: SEPARATED
ICD code: 427.31, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Atrial fibrillation
ICD code: 403.90, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hy kid NOS w cr kid I-IV
ICD code: 585.9, chapter number: 10, chapter name: Diseases of the Genitourinary System, code title: Chronic kidney dis NOS
ICD code: 403.91, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hyp kid NOS w cr kid V
ICD code: 401.9, chapter number: 7, chapter name: Diseases of the Circulatory System, code title: Hypertension NOS
ICD code: 285.9, chapter number: 4, chapter name: Diseases of the Blood and Blood-forming Organs, code title: Anemia NOS
ICD code: 428.